In [2]:
import polars as pl
from feature_store import Client
from feature_store.feature import FeatureKind
import sqlalchemy as sa

In [ ]:
# Data found here: https://www.kaggle.com/datasets/kapturovalexander/karpovs-sql-simulator

In [16]:
listings = pl.read_csv("denmark_hovedstaden_copenhagen_2022-09-24_visualisations_listings.csv").with_columns(pl.lit('2022-09-24').str.strptime(pl.Date, '%Y-%m-%d').alias('date'))

In [17]:
listings

id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license,date
i64,str,i64,str,str,str,f64,f64,str,i64,i64,i64,str,f64,i64,i64,i64,str,date
6983,"""Copenhagen 'N ...",16774,"""Simon""",null,"""Nrrebro""",55.68641,12.54741,"""Entire home/ap...",898,3,172,"""2022-06-21""",1.08,1,0,4,null,2022-09-24
26057,"""Lovely house -...",109777,"""Kari""",null,"""Indre By""",55.69307,12.57649,"""Entire home/ap...",2600,4,59,"""2022-08-09""",0.55,1,303,8,null,2022-09-24
26473,"""City Centre To...",112210,"""Julia""",null,"""Indre By""",55.67602,12.5754,"""Entire home/ap...",3250,3,300,"""2022-09-10""",2.06,3,56,7,null,2022-09-24
29118,"""Best Location ...",125230,"""Nana""",null,"""Vesterbro-Kong...",55.67023,12.55504,"""Entire home/ap...",725,7,24,"""2022-08-04""",0.16,1,59,2,null,2022-09-24
31094,"""Beautiful, spa...",129976,"""Ebbe""",null,"""Vesterbro-Kong...",55.666602,12.555283,"""Entire home/ap...",1954,3,19,"""2022-08-22""",0.13,1,0,2,null,2022-09-24
32379,"""165 m2 artist...",140105,"""Lise""",null,"""Vesterbro-Kong...",55.672638,12.552493,"""Entire home/ap...",1280,3,80,"""2022-08-13""",0.54,2,60,5,null,2022-09-24
32841,"""Cozy flat for ...",142143,"""Anders & Maria...",null,"""sterbro""",55.71176,12.57091,"""Entire home/ap...",617,100,7,"""2016-09-15""",0.05,1,281,0,null,2022-09-24
33680,"""Best location ...",145671,"""Mette""",null,"""Vesterbro-Kong...",55.66631,12.54555,"""Entire home/ap...",1000,6,71,"""2019-03-14""",0.48,1,312,0,null,2022-09-24
37159,"""Unique space o...",160390,"""Jeanette""",null,"""Indre By""",55.68547,12.56543,"""Entire home/ap...",2916,5,11,"""2017-08-22""",0.07,1,0,0,null,2022-09-24


In [26]:
neighbourhood_df = listings.select(pl.col('neighbourhood'), pl.col('date'), pl.col('id'))

In [5]:
engine = sa.create_engine("sqlite:///test.db")

In [6]:
client = Client()

In [27]:
neighbourhood_feature = client.register_feature(
    feature_name="neighbourhood",
    kind=FeatureKind.sql,
    location="main.neighbourhood",
    id_column="id",
    date_column="date",
    auth_key="local_sqlite"
)

In [28]:
client.upload_feature_data(neighbourhood_feature.name, neighbourhood_df.to_pandas())

<Feature neighbourhood>

In [29]:
client.get_available_features()

[<Feature neighborhood>, <Feature neighbourhood>]

In [30]:
f = client.get_feature("neighbourhood")

In [31]:
f.to_pandas()

,id,date,neighbourhood
0,6983,2022-09-24,Nrrebro
1,26057,2022-09-24,Indre By
2,26473,2022-09-24,Indre By
3,29118,2022-09-24,Vesterbro-Kongens Enghave
4,31094,2022-09-24,Vesterbro-Kongens Enghave
...,...,...,...
13810,653494030951422457,2022-09-24,sterbro
13811,647809303952891559,2022-09-24,Valby
13812,650819220455514228,2022-09-24,Valby
13813,658612163688161695,2022-09-24,Brnshj-Husum
